# Snake-RL (Pytorch)
Gait Decomposition을 활용하여 Gait 자체를 학습하는 연구를 이 Jupyter 노트북에 작성을 하려고함. Gait Decomposition은 뱀 로봇의 움직임을 P함수와 M행렬로 분해하여 뱀 로봇의 Gait를 표현하는 방법이며, 이 방법을 통해서 직관적으로 뱀 로봇의 움직임을 표현할 수 있고, 파라미터 튜닝을 진행할 수 있다. 관련 내용은 [Arxiv 논문](https://arxiv.org/abs/2112.02057)을 참조할 것.

먼저 Gait 생성 코드를 코드에 import하자.

In [7]:
from gait import gait

g = gait(1)

위 코드에서 삽입된 gait.py코드는 위 논문의 P함수와 M행렬을 객체로 구현한 코드이다.

Gait가 잘 생성되는지 확인해보자.

In [9]:
for i in range(0,10):
    print(g.generate(i).T)

[[15.  0. -0. -0.  0.  0. -0. -0.  0.  0.  0.  0. -0. -0.]]
[[ 0.   13.97 -0.   -0.    0.    0.   -0.   -0.    0.    0.   -0.   -0.
  -0.   -0.  ]]
[[  0.     0.   -23.31  -0.     0.     0.    -0.    -0.     0.     0.
   -0.    -0.    -0.    -0.  ]]
[[  0.     0.    -0.   -24.04   0.     0.    -0.    -0.     0.     0.
   -0.    -0.    -0.    -0.  ]]
[[ 0.    0.   -0.   -0.   28.53  0.   -0.   -0.    0.    0.   -0.   -0.
  -0.   -0.  ]]
[[ 0.    0.   -0.   -0.    0.   29.42 -0.   -0.    0.    0.   -0.   -0.
  -0.   -0.  ]]
[[  0.     0.    -0.    -0.     0.     0.   -29.96  -0.     0.     0.
   -0.    -0.    -0.    -0.  ]]
[[ -0.     0.    -0.    -0.     0.     0.    -0.   -29.08   0.     0.
   -0.    -0.     0.    -0.  ]]
[[-0.    0.   -0.   -0.    0.    0.   -0.   -0.   27.41  0.   -0.   -0.
   0.   -0.  ]]
[[-0.    0.   -0.   -0.    0.    0.   -0.   -0.    0.   23.07 -0.   -0.
   0.   -0.  ]]


위와 같이 생성된 Gait 데이터를 통해서 모터를 동작시키면 뱀 로봇이 움직이게 된다. Mujoco 시뮬레이터에서 움직이는 것을 그림으로 나타내면 아래 그림과 같다.

![fig1](./img/fig1.gif)

위 출력 결과와 같이 `i`가 증가하면서 각 모터에 입력되는 입력 신호가 바뀌는 것을 확인할 수 있다.
위 결과는 P함수와 M행렬의 행렬 곱에서 대각 원소의 값을 출력한 결과이다. 위와 같은 결과를 모터에 전달하여 뱀 로봇이 움직이도록 만들 수 있다.

P함수는 모터의 목표 위치를 계산하는 함수로 주기 함수의 꼴을 갖는다. 주변 환경이 변하지 않는 이상적인 환경에서는 미리 학습된 P함수를 통해서 뱀 로봇을 움직이게 만들 수 있다.
하지만, 뱀 로봇이 항상 이상적인 환경에서 동작하지 않기 때문에 학습된 P함수가 최적 값을 내지못할 수도 있다. 우리는 이런 점에서 P함수를 센싱되는 지형의 데이터나 로봇의 상태에 따라서 변경할 수 있는 알고리즘을 구현하고자 한다.

이를 위해 P함수의 정의를 명확하게 하고자한다.
$$
P(t,A,\theta)
$$